In [1]:
# !/usr/bin/env python
# coding: utf-8
import logging
import random
import os
import csv
import numpy as np
import pandas as pd
import datetime 
import sys
import time
import threading
import concurrent.futures
import sqlite3

from decimal import Decimal as D, ROUND_UP, getcontext

from gate_api import ApiClient, Configuration, Loan, MarginApi, Order, RepayRequest, SpotApi, Transfer, WalletApi
from gate_api.exceptions import GateApiException

In [2]:
class Accumerate_data :
    
    config = Configuration(key="", secret="")
    spot_api = SpotApi(ApiClient(config))

    def __init__(self, ticker, interval= '1d'):
        self.ticker = ticker
        self.interval = str(interval)
        current_path = os.path.dirname('__file__')
        self.home = os.path.join( current_path, 'gate_Dt' ) 
        if not os.path.exists( os.path.join( self.home, interval) ): os.makedirs( os.path.join( self.home, interval) )

    def _set_data(self, ticker, interval):
        con = sqlite3.connect( os.path.join( self.home , interval, "data.db") )
        cur = con.cursor()
        try :
            query = cur.execute("SELECT * FROM %s ORDER BY date desc LIMIT 1;" % ( ('\'' + ticker + '\'')  ))
            before = query.fetchall()[0][0]
        except :
            before = None
        yesterday = datetime.datetime.today() - datetime.timedelta(60)
        yesterday = int ( time.mktime ( yesterday.timetuple() ) )
        print(yesterday)
        current  = self.spot_api.kline_list_candletrade(currency_pair = ticker, interval = interval, _from = yesterday)
        print( "current len " , len(current))
        return before, current

    @classmethod
    def rebuild_data(cls, data):
        # 'time'  '총 거래 대금' '종가' '고가' 저가' '시작가'
        #['1636070400', '162518.88280499349', '3.695', '4.35', '3', '4.184']
        data = pd.DataFrame( data = 
                                        {
                                            'date'                  : [ float(i[0 ]) for i in data] ,
                                            'Volume'                : [ float(i[1 ]) for i in data] ,
                                            'close'                 : [ float(i[2 ]) for i in data] ,
                                            'High'                  : [ float(i[3 ]) for i in data] , 
                                            'Low'                   : [ float(i[4 ]) for i in data] ,
                                            'open'                  : [ float(i[5 ]) for i in data] ,
                                        }
                                   )
        data = data.set_index('date')
        return data

    @staticmethod
    def printALL(data):
        print( 'data', len(data)   )
        print( data[-1])

    def cmp_time_Data(self, before, current, interval, ticker):
        # try :
        last_time = datetime.datetime.fromtimestamp( before )
        Add_Time  = datetime.datetime.fromtimestamp( current.index[-1] )
        now       = datetime.datetime.now()

        print('last_time',last_time, 'Add_Time', Add_Time)
        print(interval)

        if 'm' in self.interval :
            minutes_rest = now.minute % int(self.interval.replace('m',''))
            now = now - datetime.timedelta(minutes=minutes_rest) 
            if Add_Time > last_time and Add_Time.hour == now.hour and Add_Time.minute == now.minute and Add_Time.day == now.day :
                print("save new Data")
                self.writeFile(interval, before, current.iloc[: -1], ticker)
                return True
        elif 'h' in self.interval :
            hour_rest = now.hour % int(self.interval.replace('h',''))
            now = now - datetime.timedelta(hours=hour_rest) 
            if Add_Time > last_time and Add_Time.hour == now.hour and Add_Time.day == now.day :
                print("save new Data")
                self.writeFile(interval, before, current.iloc[: -1], ticker)
                return True
        elif 'd' in self.interval :
            day_rest = now.day % int(self.interval.replace('d',''))
            now = now - datetime.timedelta(days=day_rest) 
            if Add_Time > last_time and Add_Time.day == now.day :
                print("save new Data")
                self.writeFile(interval, before, current.iloc[: -1], ticker)
                return True

        if Add_Time > last_time :
            print("시간은 같지만 현재 최근값 ")
            self.writeFile(interval, before, current , ticker)
            return True

        return False

    def writeFile(self, interval, before, current, ticker):
        print(ticker)
        con = sqlite3.connect( os.path.join( self.home , interval, "data.db") )
        last_time = datetime.datetime.fromtimestamp( before ) 
        for i in range(0, len(current)) :
            new = datetime.datetime.fromtimestamp( current.index[i] )  
            if new > last_time :
                # print(current.iloc[i:])
                current.iloc[i:].astype('float32').to_sql(ticker, con, if_exists='append')
                break

    def NewFile(self, interval, current, ticker):
        print("new data save")
        con = sqlite3.connect( os.path.join( self.home , interval, "data.db") )
        current.iloc[: -1].astype('float32').to_sql(ticker, con, if_exists='append')
        return True

    def write_real_data(self, ticker, interval):
        before, current = self._set_data(ticker, self.interval)
        current = self.rebuild_data(current)
        if before == None :
            return self.NewFile(interval, current, ticker)
        return self.cmp_time_Data(before, current, self.interval, ticker )

    def make_ticker(self, Market_Coin) :
        trade_coin = []
        for _ticker in Market_Coin : 
            print(_ticker)
            while 1 :
                try :
                    self.write_real_data(_ticker, self.interval )
                    trade_coin.append(_ticker)
                    break
                except :
                    print("wait a moment")
                    time.sleep(10)
                    self.write_real_data(_ticker, self.interval )
                    trade_coin.append(_ticker)
                    break

        return trade_coin

    def start(self):
        check_p = 0
        trade_coin = []
        trade_coin = self.make_ticker(self.ticker)
        return trade_coin

In [10]:
def ts_to_date(_date):
    _date = datetime.datetime.fromtimestamp(_date)
    format = '%Y-%m-%d %H:%M:%S'
    str_datetime = datetime.datetime.strftime(_date,format)
    return str_datetime

def rebuild_data( data):
    # 'time'  '총 거래 대금' '종가' '고가' 저가' '시작가'
    #['1636070400', '162518.88280499349', '3.695', '4.35', '3', '4.184']
    data = pd.DataFrame( data = 
                                    {
                                        'date'                  : [ ts_to_date(float(i[0 ]))  for i in data] ,
                                        'Volume'                : [ float(i[1 ]) for i in data] ,
                                        'close'                 : [ float(i[2 ]) for i in data] ,
                                        'High'                  : [ float(i[3 ]) for i in data] , 
                                        'Low'                   : [ float(i[4 ]) for i in data] ,
                                        'open'                  : [ float(i[5 ]) for i in data] ,
                                    }
                                )
    data = data.set_index('date')
    return data

In [3]:
config = Configuration(key="", secret="")
spot_api = SpotApi(ApiClient(config))
pair = spot_api.list_tickers()
ticker = []
for _pair in pair :
    if 'USDT' in _pair.currency_pair :
        ticker.append( _pair.currency_pair )
print( len( ticker ) )


1502


In [7]:
yesterday = datetime.datetime.today() - datetime.timedelta(300)
yesterday = int ( time.mktime ( yesterday.timetuple() ) )
# current  = spot_api.kline_list_candletrade(currency_pair = 'HBAR_USDT', interval = '4h', _from = yesterday)
# current = rebuild_data(current)
current = spot_api.list_candlesticks(
                currency_pair='HBAR_USDT',
                interval='4h',
                _from=yesterday,
                limit = 500
            )
current = rebuild_data(current)
print(current)

GateApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 06 Feb 2022 04:43:24 GMT', 'Content-Type': 'application/json', 'Content-Length': '121', 'Connection': 'keep-alive', 'Server': 'openresty', 'X-Request-Id': '[7ec88f17-4489354903]'})
HTTP response body: {"label":"INVALID_PARAM_VALUE","message":"Candlestick range too broad. Maximum 1000 data points are allowed per request"}


In [13]:
weekday = datetime.datetime.strptime(str('02-05'), '%m-%d').day
weekday

5

In [15]:
yesterday = datetime.datetime.today() - datetime.timedelta(1)
yesterday = int ( time.mktime ( yesterday.timetuple() ) )
current  = spot_api.kline_list_candletrade(currency_pair = 'HBAR_USDT', interval = '5m', _from = yesterday)
current = rebuild_data(current)
print(current)

                           Volume    close     High      Low     open
date                                                                 
2022-02-05 11:10:00    982.409965  0.24820  0.24820  0.24769  0.24796
2022-02-05 11:15:00  10353.557569  0.24931  0.24933  0.24823  0.24823
2022-02-05 11:20:00  27107.281660  0.25047  0.25047  0.24910  0.24910
2022-02-05 11:25:00  17479.484817  0.25136  0.25140  0.25021  0.25021
2022-02-05 11:30:00  20247.913753  0.25281  0.25298  0.25134  0.25134
...                           ...      ...      ...      ...      ...
2022-02-06 10:45:00   5554.653399  0.26026  0.26052  0.26021  0.26022
2022-02-06 10:50:00   1476.968471  0.26043  0.26043  0.26035  0.26035
2022-02-06 10:55:00      0.000000  0.26043  0.26043  0.26043  0.26043
2022-02-06 11:00:00   1494.517897  0.25929  0.26021  0.25929  0.26021
2022-02-06 11:05:00      0.000000  0.25929  0.25929  0.25929  0.25929

[288 rows x 5 columns]


In [4]:
print( ticker[0])
acc_data = Accumerate_data(ticker = [ticker[0]])
print( acc_data.start() )

RVC_USDT
RVC_USDT
1633452188
current len  60
last_time 2021-12-03 09:00:00 Add_Time 2021-12-04 09:00:00
1d
시간은 같지만 현재 최근값 
RVC_USDT
['RVC_USDT']
